In [2]:
import sys, pathlib
REPO_ROOT = pathlib.Path("/Users/Sbf/Desktop/to_verilate").resolve()
sys.path.insert(0, str(REPO_ROOT))

from emulator.emulator import QickEmulator
from qick import QickConfig
from qick.asm_v2 import AveragerProgramV2

CFG_PATH = REPO_ROOT / "emulator" / "qick_config_216.json"

In [3]:
# A) Show what’s configurable (straight from JSON/QickConfig)
soccfg = QickConfig(str(CFG_PATH))
def cfg_dict(soccfg):
    if hasattr(soccfg, "_cfg"): return soccfg._cfg
    if hasattr(soccfg, "as_dict"): 
        d = soccfg.as_dict()
        if d is not None: return d
    if hasattr(soccfg, "to_dict"):
        d = soccfg.to_dict()
        if d is not None: return d
    return {}

base = cfg_dict(soccfg)
print("gens BEFORE:", base.get("gens", []))
print("rf.tiles BEFORE:", base.get("rf", {}).get("tiles", {}))

QICK library version mismatch: 0.2.366 remote (the board), 0.2.370 local (the PC)
                        This may cause errors, usually KeyError in QickConfig initialization.
                        If this happens, you must bring your versions in sync.


gens BEFORE: [{'b_dds': 32, 'b_phase': 32, 'complex_env': True, 'dac': '00', 'f_dds': 9584.64, 'f_fabric': 599.04, 'fdds_div': 1, 'fs': 9584.64, 'fs_div': 1, 'fs_mult': 39, 'fullpath': 'axis_signal_gen_v6_0', 'has_dds': True, 'has_mixer': False, 'interpolation': 1, 'maxlen': 65536, 'maxv': 32766, 'maxv_scale': 1.0, 'mixer_freq': 100.0, 'revision': 4, 'samps_per_clk': 16, 'tproc_ch': 0, 'type': 'axis_signal_gen_v6', 'version': '1.0'}, {'b_dds': 32, 'b_phase': 32, 'complex_env': True, 'dac': '01', 'f_dds': 9584.64, 'f_fabric': 599.04, 'fdds_div': 1, 'fs': 9584.64, 'fs_div': 1, 'fs_mult': 39, 'fullpath': 'axis_signal_gen_v6_1', 'has_dds': True, 'has_mixer': False, 'interpolation': 1, 'maxlen': 16384, 'maxv': 32766, 'maxv_scale': 1.0, 'revision': 4, 'samps_per_clk': 16, 'tproc_ch': 1, 'type': 'axis_signal_gen_v6', 'version': '1.0'}, {'b_dds': 32, 'b_phase': 32, 'complex_env': True, 'dac': '02', 'f_dds': 9584.64, 'f_fabric': 599.04, 'fdds_div': 1, 'fs': 9584.64, 'fs_div': 1, 'fs_mult': 39, 

In [5]:
# B) Edit JSON via helper (this is the whole point)
emu = QickEmulator(str(CFG_PATH))
emu.set_gen_mixer(0, 100.0)      # e.g. set GEN 0 to 100 MHz
emu.set_dac_tile_fs(2, 5898.24)  # e.g. set DAC tile 2 fs
emu.commit()                     # persist and reload QickConfig

print("gens AFTER:", emu.cfg_dict().get("gens", []))
print("rf.tiles AFTER:", emu.cfg_dict().get("rf", {}).get("tiles", {}))

QICK library version mismatch: 0.2.366 remote (the board), 0.2.370 local (the PC)
                        This may cause errors, usually KeyError in QickConfig initialization.
                        If this happens, you must bring your versions in sync.
QICK library version mismatch: 0.2.366 remote (the board), 0.2.370 local (the PC)
                        This may cause errors, usually KeyError in QickConfig initialization.
                        If this happens, you must bring your versions in sync.


gens AFTER: [{'b_dds': 32, 'b_phase': 32, 'complex_env': True, 'dac': '00', 'f_dds': 9584.64, 'f_fabric': 599.04, 'fdds_div': 1, 'fs': 9584.64, 'fs_div': 1, 'fs_mult': 39, 'fullpath': 'axis_signal_gen_v6_0', 'has_dds': True, 'has_mixer': False, 'interpolation': 1, 'maxlen': 65536, 'maxv': 32766, 'maxv_scale': 1.0, 'mixer_freq': 100.0, 'revision': 4, 'samps_per_clk': 16, 'tproc_ch': 0, 'type': 'axis_signal_gen_v6', 'version': '1.0'}, {'b_dds': 32, 'b_phase': 32, 'complex_env': True, 'dac': '01', 'f_dds': 9584.64, 'f_fabric': 599.04, 'fdds_div': 1, 'fs': 9584.64, 'fs_div': 1, 'fs_mult': 39, 'fullpath': 'axis_signal_gen_v6_1', 'has_dds': True, 'has_mixer': False, 'interpolation': 1, 'maxlen': 16384, 'maxv': 32766, 'maxv_scale': 1.0, 'revision': 4, 'samps_per_clk': 16, 'tproc_ch': 1, 'type': 'axis_signal_gen_v6', 'version': '1.0'}, {'b_dds': 32, 'b_phase': 32, 'complex_env': True, 'dac': '02', 'f_dds': 9584.64, 'f_fabric': 599.04, 'fdds_div': 1, 'fs': 9584.64, 'fs_div': 1, 'fs_mult': 39, '

In [8]:
GEN_CH = 0
RO_CH = 0
TRIG_TIME = 0.40
FREQ = 100
# FREQ = 0
class MultiPulseProgram(AveragerProgramV2):
    def _initialize(self, cfg):
        ro_ch = cfg['ro_ch']
        gen_ch = cfg['gen_ch']
        
        self.declare_gen(ch=gen_ch, nqz=1)
        self.declare_readout(ch=ro_ch, length=cfg['ro_len'])

        ramp_len = self.cycles2us(20, gen_ch=gen_ch)
        self.add_gauss(ch=gen_ch, name="ramp", sigma=ramp_len/10, length=ramp_len, even_length=True)
        
        self.add_pulse(ch=gen_ch, name="myflattop", ro_ch=ro_ch, 
                       style    = "flat_top", 
                       envelope = "ramp", 
                       freq     = cfg['freq'], 
                       length   = 0.1,
                       phase    = 0,
                       gain     = 1.0, 
                      )

        self.add_pulse(ch=gen_ch, name="myflattop2", ro_ch=ro_ch, 
                       style    = "flat_top", 
                       envelope = "ramp", 
                       freq     = cfg['freq'], 
                       length   = 0.1,
                       phase    = 90,
                       gain     = 0.5, 
                      )

        self.add_readoutconfig(ch=ro_ch, name="myro", freq=cfg['freq'], gen_ch=gen_ch)
        # send the config to the dynamic RO
        self.send_readoutconfig(ch=ro_ch, name="myro", t=0)
        
    def _body(self, cfg):
        self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['trig_time'], ddr4=True)

        for i in range(0,15):
            self.pulse(ch=cfg['gen_ch'], name="myflattop", t=1.0)
            self.pulse(ch=cfg['gen_ch'], name="myflattop2", t=1.0)
config = {'gen_ch': GEN_CH,
          'ro_ch': RO_CH,
          'freq': FREQ,
          'trig_time': 1.0,
          'ro_len': 5.0,
         }
prog = MultiPulseProgram(soccfg, reps=1, final_delay=0.5, cfg=config)
emu.export_pmem_hex(prog, path=REPO_ROOT/"emulator/pmem.mem")
emu.export_wmem_hex(prog, stem=str(REPO_ROOT/"emulator/wmem"))  # lane files or single, depending on QICK
emu.export_sgmem(prog, ch=0, prefix=str(REPO_ROOT/"emulator/sgmem_ch"))

In [9]:
prog.print_pmem2hex()

// PMEM content
000000000000000000
8c600000000000000c
dce000c20000000000
4c080000000000d700
8c6000000000000020
dda00030000000d700
dda00034800000d700
dda00035000000d700
dda00010000000dc00
dda00014800000dc00
dda00015000000dc00
dce00000000000d700
dce00020000000d700
dce00040000000d700
dce00060000000d700
dce00080000000d700
dce000a0000000d700
dce00000000000d700
dce00020000000d700
dce00040000000d700
dce00060000000d700
dce00080000000d700
dce000a0000000d700
dce00000000000d700
dce00020000000d700
dce00040000000d700
dce00060000000d700
dce00080000000d700
dce000a0000000d700
dce00000000000d700
dce00020000000d700
dce00040000000d700
dce00060000000d700
dce00080000000d700
dce000a0000000d700
dce00000000000d700
dce00020000000d700
dce00040000000d700
dce00060000000d700
dce00080000000d700
dce000a0000000d700
dce00000000000d700
dce00020000000d700
dce00040000000d700
dce00060000000d700
dce00080000000d700
dce000a0000000d700
dce00000000000d700
dce00020000000d700
dce00040000000d700
dce00060000000d700
dce00080000000d

In [1]:
import sys, importlib, pathlib, os

REPO_ROOT = pathlib.Path("/Users/Sbf/Desktop/to_verilate").resolve()
PKG_DIR   = REPO_ROOT / "emulator"               
INIT_FILE = PKG_DIR / "__init__.py"

# 1) Ensure package marker exists
if not INIT_FILE.exists():
    INIT_FILE.write_text("from .emulator import QickEmulator\n")

# 2) Clean bad sys.path entries that point *inside* the package dir
sys.path = [p for p in sys.path
            if not (pathlib.Path(p).exists() and pathlib.Path(p).resolve() == PKG_DIR.resolve())]

# 3) Ensure only the repo root is on sys.path (so 'emulator' is a package)
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

# 4) Nuke any previously imported shadow module and reload finders
sys.modules.pop("emulator", None)
importlib.invalidate_caches()

# 5) Now this will work (because emulator is a package)
from emulator.emulator import QickEmulator
print("Imported QickEmulator OK")

Imported QickEmulator OK


/Users/sbf/Desktop/to_verilate/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
